In [215]:
import pandas as pd
import numpy as np

In [216]:
data = pd.read_csv('gym_data.csv')

In [217]:
data.drop(columns = ['Unnamed: 25','User'],inplace = True)

In [218]:
data['Category 11'][2712]= 2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [219]:
data['Category 11'] = pd.to_numeric(data['Category 11'])

In [220]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5456 entries, 0 to 5455
Data columns (total 24 columns):
Category 1     5456 non-null float64
Category 2     5456 non-null float64
Category 3     5456 non-null float64
Category 4     5456 non-null float64
Category 5     5456 non-null float64
Category 6     5456 non-null float64
Category 7     5456 non-null float64
Category 8     5456 non-null float64
Category 9     5456 non-null float64
Category 10    5456 non-null float64
Category 11    5456 non-null float64
Category 12    5455 non-null float64
Category 13    5456 non-null float64
Category 14    5456 non-null float64
Category 15    5456 non-null float64
Category 16    5456 non-null float64
Category 17    5456 non-null float64
Category 18    5456 non-null float64
Category 19    5456 non-null float64
Category 20    5456 non-null float64
Category 21    5456 non-null float64
Category 22    5456 non-null float64
Category 23    5456 non-null float64
Category 24    5455 non-null float64
dtype

In [221]:
data['averge'] =np.round((data.sum(axis = 1))/(len(data.columns)),1)

### By considering  the mean of all categories as the satisfaction level for joining or taking the gym membership we are preparing the model based on this condition

In [222]:
data['membership'] = data.averge.map(lambda d: "Yes" if d > data.averge.mean() else "No")

### Base line model without pipeline and any parameter tuning

In [223]:
from sklearn.model_selection import train_test_split

In [224]:
from sklearn.preprocessing import LabelEncoder

In [225]:
data['membership_data'] = LabelEncoder().fit_transform(target_data)

In [226]:
data.columns

Index(['Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5',
       'Category 6', 'Category 7', 'Category 8', 'Category 9', 'Category 10',
       'Category 11', 'Category 12', 'Category 13', 'Category 14',
       'Category 15', 'Category 16', 'Category 17', 'Category 18',
       'Category 19', 'Category 20', 'Category 21', 'Category 22',
       'Category 23', 'Category 24', 'averge', 'membership',
       'membership_data'],
      dtype='object')

In [227]:
feature_data =  data.drop(columns = ['membership','membership_data','averge'])

In [243]:
feature_data.fillna(data.averge.mean(),inplace = True)

In [245]:
target_data  = data.membership_data

In [246]:
trainX, testX, trainY, testY = train_test_split(feature_data, target_data)

In [247]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [248]:
feature_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5456 entries, 0 to 5455
Data columns (total 24 columns):
Category 1     5456 non-null float64
Category 2     5456 non-null float64
Category 3     5456 non-null float64
Category 4     5456 non-null float64
Category 5     5456 non-null float64
Category 6     5456 non-null float64
Category 7     5456 non-null float64
Category 8     5456 non-null float64
Category 9     5456 non-null float64
Category 10    5456 non-null float64
Category 11    5456 non-null float64
Category 12    5456 non-null float64
Category 13    5456 non-null float64
Category 14    5456 non-null float64
Category 15    5456 non-null float64
Category 16    5456 non-null float64
Category 17    5456 non-null float64
Category 18    5456 non-null float64
Category 19    5456 non-null float64
Category 20    5456 non-null float64
Category 21    5456 non-null float64
Category 22    5456 non-null float64
Category 23    5456 non-null float64
Category 24    5456 non-null float64
dtype

In [249]:
feature_data.isna().sum()

Category 1     0
Category 2     0
Category 3     0
Category 4     0
Category 5     0
Category 6     0
Category 7     0
Category 8     0
Category 9     0
Category 10    0
Category 11    0
Category 12    0
Category 13    0
Category 14    0
Category 15    0
Category 16    0
Category 17    0
Category 18    0
Category 19    0
Category 20    0
Category 21    0
Category 22    0
Category 23    0
Category 24    0
dtype: int64

In [250]:
names = ['linear_model','logistic_model','Decision_tree','Naive_bayes','randomforest','xgbclassifier']
models =  [LinearRegression(),LogisticRegression(),DecisionTreeClassifier(),GaussianNB(),RandomForestClassifier(),XGBClassifier()]
for name, estimator in zip(names, models):
    estimator.fit(trainX,trainY)
    print(estimator.score(testX,testY))

0.6351263454540195


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9714076246334311
0.8357771260997068
0.7265395894428153


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8673020527859238
0.9288856304985337


### Feature_selection 

In [268]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import f_classif, SelectKBest

In [259]:
float_data = feature_data.select_dtypes(include = ['float'])

In [260]:
int_data = feature_data.select_dtypes(include=['int64'])

In [261]:
from sklearn.preprocessing import MinMaxScaler

In [263]:
float_pipeline = make_pipeline(MinMaxScaler())
int_pipeline = make_pipeline(MinMaxScaler())

In [264]:
from sklearn.compose import make_column_transformer

In [266]:
preprocessor = make_column_transformer(
(int_pipeline, int_data.columns),(float_pipeline, float_data.columns), remainder = 'passthrough'
)

In [267]:
from sklearn.preprocessing import StandardScaler


In [270]:
pipeline = make_pipeline(StandardScaler(), SelectKBest(k = 10, score_func = f_classif), RandomForestClassifier(n_estimators = 100))

In [273]:
pipeline.fit(trainX, trainY)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('selectkbest',
                 SelectKBest(k=10,
                             score_func=<function f_classif at 0x000000E1279ACCA8>)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_sc

In [274]:
pipeline.score(testX, testY)

0.8570381231671554

In [277]:
names = ['linear_model','logistic_model','Decision_tree','Naive_bayes','randomforest','xgbclassifier']
models =  [LinearRegression(),LogisticRegression(),DecisionTreeClassifier(),GaussianNB(),RandomForestClassifier(n_estimators= 100),XGBClassifier()]
for name, estimator in zip(names, models):
    pipeline = make_pipeline(StandardScaler(), SelectKBest(k = 10, score_func = f_classif), estimator)
    pipeline.fit(trainX,trainY)
    print(name,pipeline.score(testX,testY))

linear_model 0.2742948743683077
logistic_model 0.7280058651026393


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Decision_tree 0.7998533724340176
Naive_bayes 0.6796187683284457
randomforest 0.8658357771260997
xgbclassifier 0.8548387096774194
